In [19]:
from openai import OpenAI
from sk import OPENAI_API_KEY
import time
client = OpenAI(api_key=OPENAI_API_KEY)



In [20]:
# cannot create properly, create assistant manualy now.
# Few-shot Prompting can be added
intstructions_string = """
    ShenNutritionGPT can provide dietary recommendations \
    based on your nutritional needs through your food preferences. \
    In addition to recommending basic ingredients, it can also suggest \
    suitable options for dining out and prepare recipes for you, \
    making it much easier for you to control your diet.
    
    Here are examples of ShawGPT responding to viewer comments.

    Viewer comment: Do you have any food suggestions? I have to eat 20g of protein this meal.
    ShenNutritionGPT: Great to hear, glad it was helpful :) -ShenNutritionGPT

    Viewer comment: Epic, very useful for my BCI class
    ShenNutritionGPT: Thanks, glad to hear! -ShenNutritionGPT

    Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Shaw. Thank you. It's so rare to find good communicators like you!
    ShenNutritionGPT: Thanks, glad it was clear -ShenNutritionGPT
    """

# file = client.files.create(
#   file=open("gpt_assistant_input/foundation_food_nutrition.pdf", "rb"),
#   purpose="assistants"
# )

# assistant = client.beta.assistants.create(
#     name="ShenNutritionGPT",
#     description="Food Nutrition GPT for Diet Recommendation",
#     instructions=intstructions_string,
#     tools=[{"type": "retrieval"}],
#     file_ids=[file.id],
#     model="gpt-3.5-turbo-0125"
# )
assistant = client.beta.assistants.retrieve("asst_Z4knqwOe9olvisDtzZGP0Fxo")

In [21]:
# create thread (i.e. object that handles conversation between user and assistant)
thread = client.beta.threads.create()

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need 20g of protein. Do you have any food suggestions?"
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Waits for the run to be completed. 
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, 
                                                   run_id=run.id)
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        print("Run failed:", run_status.last_error)
        break
    time.sleep(2)  # wait for 2 seconds before checking again

In [22]:
# Step 5: Parse the Assistant's Response to Print the Results
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Prints the messages with the latest message at the bottom
number_of_messages = len(messages.data)
print( f'Number of messages: {number_of_messages}')

for message in reversed(messages.data):
    role = message.role  
    for content in message.content:
        if content.type == 'text':
            response = content.text.value 
            print(f'\n{role}: {response}')

Number of messages: 2

user: I need 20g of protein. Do you have any food suggestions?

assistant: To meet your requirement of 20g of protein, you can consider the following options from the list of food items provided in the document:

1. Chicken breast boneless skinless raw: Provides 19.17g of protein
2. Beans black canned sodium added drained and rinsed: Provides 6.91g of protein
3. Beans cannellini canned sodium added drained and rinsed: Provides 7.41g of protein

These are some of the options that can help you reach your desired protein intake. Feel free to combine them or choose one based on your preferences.
